# Transfer Learning (VGG16)

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import time
import shutil

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.enable_eager_execution()

os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Usage `VGG16`
* [code link](https://github.com/keras-team/keras-applications/blob/master/keras_applications/vgg16.py)
* [document link](https://keras.io/applications/#vgg16)

```python
def VGG16(include_top=True,
          weights='imagenet',
          input_tensor=None,
          input_shape=None,
          pooling=None,
          classes=1000,
          **kwargs):
    """Instantiates the VGG16 architecture.
    Optionally loads weights pre-trained on ImageNet.
    Note that the data format convention used by the model is
    the one specified in your Keras config at `~/.keras/keras.json`.
    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor
            (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)`
            (with `channels_last` data format)
            or `(3, 224, 224)` (with `channels_first` data format).
            It should have exactly 3 input channels,
            and width and height should be no smaller than 32.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional block.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional block, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
```

### Load VGG16 conv base

In [ ]:
conv_base = tf.keras.applications.VGG16(weights='imagenet',
                                        include_top=False,
                                        input_shape=(150, 150, 3))

In [ ]:
conv_base.summary()

### Build a our model based on VGG16 conv base

In [ ]:
model = tf.keras.Sequential()

In [ ]:
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

In [ ]:
model.summary()

### Freeze conv base

In [ ]:
# assign conv1_1 and dense1 weight to compare itself after training some steps
for var in model.variables:
  #print(var.name)
  if var.name == 'block1_conv1/kernel:0':
    conv1_1_w = var
  if var.name == 'dense/kernel:0':
    dense_w = var

In [ ]:
print(conv1_1_w)

In [ ]:
print(dense_w)

In [ ]:
# Freeze vgg16 conv base part (means that trainable option is False)
for layer in model.layers:
  if layer.name == 'vgg16':
    layer.trainable = False
  print("variable name: {}, trainable: {}".format(layer.name, layer.trainable))

## Load a Google Flower Dataset

In [ ]:
# I upload zip file on my dropbox
# if you want to download from my dropbox uncomment below  
DATASET_PATH='../../datasets/flower'
  
if not os.path.isdir(DATASET_PATH):
  os.makedirs(DATASET_PATH)
  
  import urllib.request
  u = urllib.request.urlopen(url='https://www.dropbox.com/s/1tqczockfgdnz8z/flower.zip?dl=1')
  data = u.read()
  u.close()
 
  with open('flower.zip', "wb") as f :
    f.write(data)
  print('Data has been downloaded')
  
  shutil.move(os.path.join('flower.zip'), os.path.join(DATASET_PATH))
  file_path = os.path.join(DATASET_PATH, 'flower.zip')
  
  import zipfile
  zip_ref = zipfile.ZipFile(file_path, 'r')
  zip_ref.extractall(DATASET_PATH)
  zip_ref.close()
  print('Data has been extracted.')
  
else:
  print('Data has already been downloaded and extracted.')

In [ ]:
base_dir = os.path.join(DATASET_PATH, 'flower')

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

print(train_dir)
print(validation_dir)
print(test_dir)

In [ ]:
class_name = sorted(os.listdir(train_dir))
for name in class_name:
  print(name)

In [ ]:
num_train = 0
num_val = 0
num_test = 0
for name in class_name:
  train_path = os.path.join(train_dir, name)
  val_path = os.path.join(validation_dir, name)
  test_path = os.path.join(test_dir, name)
  print("Number of {} class: for train: {} / for validation: {} / for test: {}".format(name,
                                                                len(os.listdir(train_path)),
                                                                len(os.listdir(val_path)),
                                                                len(os.listdir(train_path))))
  num_train += len(os.listdir(train_path))
  num_val += len(os.listdir(val_path))
  num_test += len(os.listdir(test_path))

print('--------')
print("Total training images:", num_train)
print("Total validation images:", num_val)
print("Total test images:", num_test)

## Set hyperparameters

In [ ]:
batch_size = 16

## Setting image generator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

In [ ]:
val_generator = val_datagen.flow_from_directory(validation_dir,
                                                target_size=(150, 150),
                                                batch_size=batch_size,
                                                class_mode='categorical')

In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150, 150),
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

## Compile a model

In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=int(np.ceil(num_train / float(batch_size))),
    epochs=10,
    validation_data=val_generator,
    validation_steps=int(np.ceil(num_val / float(batch_size)))
)

In [ ]:
# Compare weights between before and after training some steps
for var in model.variables:
  #print(var.name)
  if var.name == 'block1_conv1/kernel:0':
    conv1_1_w_1 = var
  if var.name == 'dense/kernel:0':
    dense_w_1 = var

### Visualizing results of the training

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Evaluation for Test dataset

In [ ]:
history = model.evaluate_generator(test_generator)

In [ ]:
# loss
print("loss value: {:.3f}".format(history[0]))
# accuracy
print("accuracy value: {:.3f}".format(history[1]))